This is the notebook we used for using the same network architecture to predict the correct paragraph.(we didn't use this approach in the end due to performance issue)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.python.client import timeline
import json
import pickle
import math

filename = "./record/train.tfrecords"

epoch = 1000
batch = 32
classes = 2

In [ ]:
embedding = np.load("embedding.npy").astype("float32")
with open("word_dict.json","r") as f:
    word_dict = json.load(f)

In [ ]:
c = np.load("c.npy")
c_char = np.load("c_char.npy")
q = np.load("q.npy")
q_char = np.load("q_char.npy")
dlab = np.load("lab.npy")

In [ ]:
def unknown_detection(token_list):
    new_list = []
    for token in token_list:
        if token in word_dict:
            new_list.append(token)
        else:
            new_list.append("<UNK>")
    return new_list

In [ ]:
train_indices = np.arange(len(c))
def dev_batch(batch=64):
    while True:
        np.random.shuffle(train_indices)
        for i in range(int(math.ceil(len(c)/batch))):
            start_index = (i*batch)%len(c)
            idx = train_indices[start_index:start_index+batch]
            c_b = c[idx]
            c_char_b = c_char[idx]
            q_b = q[idx]
            q_char_b = q_char[idx]
            dlab_b = dlab[idx]
            yield c_b, c_char_b, q_b, q_char_b, dlab_b

In [ ]:
def prob_dp(set1,set2):
    assert len(set1) == len(set2)
    max1 = 0
    maxi1 = 0
    maxpair = None
    maxp = 0
    for i in range(len(set1)):
        if set1[i]>max1:
            max1 = set1[i]
            maxi1 = i
        if max1 * set2[i] > maxp:
            maxp = max1 * set2[i]
            maxpair = [maxi1,i]
    assert maxpair[0] <= maxpair[1]
    return maxpair,maxp

def f_train(pred_s, pred_e, true_s, true_e, context):
    # computes average f_measure for a batch
    f_sum = 0
    l = len(pred_s)
    for i in range(l):
        #s_i = np.argmax(pred_s[i])
        #e_i = np.argmax(pred_e[i])
        
        [s_i,e_i],_ = prob_dp(pred_s[i],pred_e[i])
        if e_i < s_i:
            continue
        TP, FN, FP = 0, 0, 0
        guess = context[i][s_i:e_i+1]
        true = context[i][true_s[i]:true_e[i]+1]
        for token in guess:
            if token in true:
                TP += 1
            else:
                FP += 1
        for token in true:
            if token not in guess:
                FN += 1
        precision = TP/(TP+FP)
        recall = TP/(TP+FN)
        f = 2*precision*recall/(precision+recall+1e-8)
        f_sum += f
    return f_sum/l

def f_dev(pred_s, pred_e, a, context):
    # computes average f_measure for a batch
    f_sum = 0
    l = len(pred_s)
    for i in range(l):
        #s_i = np.argmax(pred_s[i])
        #e_i = np.argmax(pred_e[i])
        
        [s_i,e_i],_ = prob_dp(pred_s[i],pred_e[i])
        if e_i < s_i:
            continue
        TP, FN, FP = 0, 0, 0
        guess = context[i][s_i:e_i+1]
        true = [word_dict[t] for t in unknown_detection(a[i])]
        for token in guess:
            if token in true:
                TP += 1
            else:
                FP += 1
        for token in true:
            if token not in guess:
                FN += 1
        precision = TP/(TP+FP)
        recall = TP/(TP+FN)
        f = 2*precision*recall/(precision+recall+1e-8)
        f_sum += f
    return f_sum/l

In [ ]:
def layer_softmax(logits,axis=-1):
    #return tf.keras.backend.softmax(logits,axis=axis)
    #return tf.nn.softmax(logits,axis=axis)
    return tf.keras.layers.Softmax(axis=axis)(logits)
    return tf.exp(logits)/tf.reduce_sum(tf.exp(logits),axis = axis,keepdims = True)

def embedding_encoder_block(scope, inputs):
    with tf.variable_scope(scope,reuse=tf.AUTO_REUSE):
        # first encode input with position info
        pos_encoded = position_encoding(inputs)
        # project input to dimension 128
        residual1 = tf.layers.separable_conv1d(pos_encoded, 128, 1, padding="same",activation=tf.nn.relu)
        
        #convolution block
        norm1 = tf.contrib.layers.layer_norm(residual1)
        norm1 = tf.nn.dropout(norm1, 1-dp)
        conv1 = tf.layers.separable_conv1d(norm1, 128, 7, padding="same",activation=tf.nn.relu)
        conv1 = tf.nn.dropout(conv1, 1-dp)
        residual2 = tf.add(residual1, conv1)
        
        norm2 = tf.contrib.layers.layer_norm(residual2)
        norm2 = tf.nn.dropout(norm2, 1-dp)
        conv2 = tf.layers.separable_conv1d(norm2, 128, 7, padding="same",activation=tf.nn.relu)
        conv2 = tf.nn.dropout(conv2, 1-dp)
        residual3 = tf.add(residual2, conv2)
        
        norm3 = tf.contrib.layers.layer_norm(residual3)
        norm3 = tf.nn.dropout(norm3, 1-dp)
        conv3 = tf.layers.separable_conv1d(norm3, 128, 7, padding="same",activation=tf.nn.relu)
        conv3 = tf.nn.dropout(conv3, 1-dp)
        residual4 = tf.add(residual3, conv3)
        
        norm4 = tf.contrib.layers.layer_norm(residual4)
        norm4 = tf.nn.dropout(norm4, 1-dp)
        conv4 = tf.layers.separable_conv1d(norm4, 128, 7, padding="same",activation=tf.nn.relu)
        conv4 = tf.nn.dropout(conv4, 1-dp)
        residual5 = tf.add(residual4, conv4)

        # self-attention block
        norm4 = tf.contrib.layers.layer_norm(residual5)
        attention = tf.matmul(norm4, norm4, transpose_b=True)
        dk = tf.cast(tf.shape(norm4)[-1], dtype=tf.float32)
        scaled = tf.divide(attention, tf.sqrt(dk))
        attention = layer_softmax(scaled, axis=-1)
        #attention = tf.nn.softmax(scaled, axis=-1)
        attention_out = tf.matmul(attention, norm4)
        residual6 = tf.add(residual5, attention_out)
        
        # feedforwoad layer
        norm5 = tf.contrib.layers.layer_norm(residual6)
        norm5 = tf.nn.dropout(norm5, 1-dp)
        ffn1 = tf.layers.separable_conv1d(norm5, 128, 1, activation=tf.nn.relu)
        ffn1 = tf.nn.dropout(ffn1, 1-dp)
        ffn2 = tf.layers.separable_conv1d(ffn1, 128, 1)
        ffn2 = tf.nn.dropout(ffn2, 1-dp)
        residual7 = tf.add(residual6, ffn2)
    return residual7

def model_encoder_block(scope, inputs, projection=False):
    with tf.variable_scope(scope,reuse=tf.AUTO_REUSE):
        inputs = position_encoding(inputs)
        if projection:
            outputs = tf.layers.separable_conv1d(inputs, 128, 1, padding="same", activation=tf.nn.relu)
        else:
            outputs = inputs
        for i in range(7):
            with tf.variable_scope("conv_block{}".format(i),reuse=tf.AUTO_REUSE):
                norm0 = tf.contrib.layers.layer_norm(outputs)
                norm0 = tf.nn.dropout(norm0, 1-dp)
                conv0 = tf.layers.separable_conv1d(norm0, 128, 5, padding="same", activation=tf.nn.relu)
                conv0 = tf.nn.dropout(conv0, 1-dp)
                residual0 = tf.add(outputs, conv0)
                
                norm1 = tf.contrib.layers.layer_norm(residual0)
                norm1 = tf.nn.dropout(norm1, 1-dp)
                conv1 = tf.layers.separable_conv1d(norm1, 128, 5, padding="same", activation=tf.nn.relu)
                conv1 = tf.nn.dropout(conv1, 1-dp)
                residual1 = tf.add(residual0, conv1)
            
            with tf.variable_scope("self_attention{}".format(i),reuse=tf.AUTO_REUSE):
                norm2 = tf.contrib.layers.layer_norm(residual1)
                norm2 = tf.nn.dropout(norm2, 1-dp)
                attention_out = multihead_self_attention(norm2, "self_attention")
                attention_out = tf.nn.dropout(attention_out, 1-dp)
                residual2 = tf.add(residual1, attention_out)
            
            with tf.variable_scope("feedforward{}".format(i),reuse=tf.AUTO_REUSE):
                norm3 = tf.contrib.layers.layer_norm(residual2)
                norm3 = tf.nn.dropout(norm3, 1-dp)
                ffn1 = tf.layers.separable_conv1d(norm3, 128, 1, activation=tf.nn.relu)
                ffn1 = tf.nn.dropout(ffn1, 1-dp)
                ffn2 = tf.layers.separable_conv1d(ffn1, 128, 1)
                ffn2 = tf.nn.dropout(ffn2, 1-dp)
                outputs = tf.add(residual2, ffn2)
    return outputs

def highway(scope, inputs):
    # two layer highway network
    size = inputs.shape.as_list()[-1]
    with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
        T1 = tf.layers.separable_conv1d(inputs, size, 1, activation=tf.nn.sigmoid, bias_initializer=tf.constant_initializer(-1))
        H1 = tf.layers.separable_conv1d(inputs, size, 1, activation=tf.nn.relu)
        H1 = tf.nn.dropout(H1, 1-dp)
        highway1 = T1 * H1 + inputs * (1.0 - T1)
        
        T2 = tf.layers.separable_conv1d(highway1, size, 1, activation=tf.nn.sigmoid, bias_initializer=tf.constant_initializer(-1))
        H2 = tf.layers.separable_conv1d(highway1, size, 1, activation=tf.nn.relu)
        H2 = tf.nn.dropout(H2, 1-dp)
        highway2 = T2 * H2 + highway1 * (1.0 - T2)
    return highway2

def multihead_self_attention(inputs, scope, heads=8):
    # restricted multi-head self-attention
    with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
        depth = inputs.get_shape().as_list()[-1]
        WQ = tf.get_variable("WQ", [depth, depth])
        WK = tf.get_variable("WK", [depth, depth])
        WV = tf.get_variable("WV", [depth, depth])
        
        Q = tf.einsum("bsd,dw->bsw", inputs, WQ)
        K = tf.einsum("bsd,dw->bsw", inputs, WK)
        V = tf.einsum("bsd,dw->bsw", inputs, WV) # [batch, sequence_length, depth]
        
        # split shape for vectorization
        Q_ = tf.concat(tf.split(Q, heads, axis=2), axis=0)
        K_ = tf.concat(tf.split(K, heads, axis=2), axis=0)
        V_ = tf.concat(tf.split(V, heads, axis=2), axis=0) # [8 * batch, sequence_length, depth / 8]
        
        attention_logits = tf.matmul(Q_, K_, transpose_b=True)
        dk = depth / heads
        scaled = tf.divide(attention_logits, tf.sqrt(dk))
        attention = layer_softmax(scaled, axis=-1)
        #attention = tf.nn.softmax(scaled, axis=-1)
        attention_out = tf.matmul(attention, V_)
        
        # retrieve shape
        attention_out = tf.concat(tf.split(attention_out, heads, axis=0), axis=2)
    return attention_out

def position_encoding(inputs):
    """
    sinusoids position encoding
    from One Model to Learn Then All
    -- input: [None, sequence_length, depth]
    -- output: [None, sequence_length, depth]
    """
    _, seq_length, depth = inputs.get_shape().as_list()
    pos_encoding = np.array([
        [pos * np.power(1e-4, -(i//2)*2/depth) for i in range(depth)]
        for pos in range(seq_length)])
    pos_encoding[:, 0::2] = np.sin(pos_encoding[:, 0::2])
    pos_encoding[:, 1::2] = np.cos(pos_encoding[:, 1::2])
    pos_encoding = tf.convert_to_tensor(pos_encoding, tf.float32)
    return inputs+pos_encoding

def query_context_co_attention(w, inputs, context, query):
    """
    input: 
        w: similarity funciton weight
        inputs: [q, c, q*c]
    output:
        A: context-to-query attention
        B: query-to-context attention
    """
    # similarity matrix S (logits)
    S = tf.einsum("abcde,ef->abcdf", tf.expand_dims(inputs,3),w)
    S = tf.squeeze(S,[-2,-1])
    
    # S_: softmax over rows
    S_ = layer_softmax(S)
    #S_ = tf.nn.softmax(S)
    
    # S__T: transpose of softmax over coloum
    S__T = tf.transpose(layer_softmax(S, axis=1),[0,2,1])
    #S__T = tf.transpose(tf.nn.softmax(S, axis=1),[0,2,1])
    
    # context_query attention
    A = tf.matmul(S_, query)
    # query_context attention
    B = tf.matmul(tf.matmul(S_, S__T), context)
    return A, B


In [ ]:
tf.reset_default_graph()

dataset = tf.data.TFRecordDataset(filename)

def parser(record):
    keys_to_features = {
        "c": tf.FixedLenSequenceFeature((), tf.int64, allow_missing=True),
        "c_char": tf.FixedLenSequenceFeature((), tf.int64, allow_missing=True),
        "q": tf.FixedLenSequenceFeature((), tf.int64, allow_missing=True),
        "q_char": tf.FixedLenSequenceFeature((), tf.int64, allow_missing=True),
        "label":tf.FixedLenFeature(shape=[], dtype=tf.int64)
    }
    parsed = tf.parse_single_example(record, keys_to_features)
    c = tf.cast(parsed["c"], tf.int32)
    c_char = tf.cast(parsed["c_char"], tf.int32)
    q = tf.cast(parsed["q"], tf.int32)
    q_char = tf.cast(parsed["q_char"], tf.int32)
    label = tf.cast(parsed["label"], tf.int32)

    return c, c_char, q, q_char, label

def make_dataset(dataset):
    dataset = dataset.map(parser)
    dataset = dataset.apply(tf.contrib.data.shuffle_and_repeat(10000, epoch))
    dataset = dataset.batch(batch)
    iterator = dataset.make_initializable_iterator()
    return iterator

train_iter = make_dataset(dataset)
next_batch = train_iter.get_next()

with tf.name_scope("inputs"):
    q_input = tf.placeholder(tf.int64, [None, 30], name="q")
    q_char_input = tf.placeholder(tf.int64, [None, 480], name="q_char")
    c_input = tf.placeholder(tf.int64, [None, 240], name="c")
    c_char_input = tf.placeholder(tf.int64, [None, 3840], name="c_char")
    
    labels = tf.placeholder(tf.int64,[None],name="para_label")
    
    start_mask = tf.placeholder(tf.int64, [None], name="start_mask")
    end_mask = tf.placeholder(tf.int64, [None], name="end_mask")
    
    batch_size = tf.placeholder(tf.int64, (), name="batch_size")
    dp = tf.placeholder(tf.float32, (), name="drop_prob")

tf.add_to_collection("infer_input", q_input)
tf.add_to_collection("infer_input", q_char_input)
tf.add_to_collection("infer_input", c_input)
tf.add_to_collection("infer_input", c_char_input)
tf.add_to_collection("infer_input", dp)

with tf.variable_scope("Input_Embedding_Layer"):
    # input embedding layer
    with tf.variable_scope("W_Embedding"):
        pretrained_embedding = tf.get_variable("w_embedding",
                                               shape=[72497,300],
                                               initializer=tf.constant_initializer(embedding),
                                               trainable=False)
        unknown_embedding = tf.get_variable("unknown",
                                            shape=[1, 300],
                                            initializer=tf.random_uniform_initializer(-0.5,0.5),
                                            trainable=True)
        tf.summary.histogram("unknown_word_embedding", unknown_embedding)
        padding_embedding = tf.get_variable("padding",
                                            shape=[1, 300],
                                            initializer=tf.zeros_initializer(),
                                            trainable=False)
        word_embedding = tf.concat([pretrained_embedding, unknown_embedding, padding_embedding], 0)
        
        q_embed = tf.nn.embedding_lookup(word_embedding, q_input)
        q_embed = tf.nn.dropout(q_embed, 1-dp)
        c_embed = tf.nn.embedding_lookup(word_embedding, c_input)
        c_embed = tf.nn.dropout(c_embed, 1-dp)

    with tf.variable_scope("C_Embedding"):
        char_embedding = tf.get_variable("c_embedding",
                                         shape=[209, 200],
                                         initializer=tf.random_uniform_initializer(-0.5,0.5),
                                         trainable=True)
        padding = tf.get_variable("padding",
                                  shape=[1, 200],
                                  initializer=tf.zeros_initializer(),
                                  trainable=False)
        char_combined = tf.concat([char_embedding, padding], 0)
        tf.summary.histogram("character_embedding", char_combined)
        q_char_embed = tf.nn.embedding_lookup(char_combined, q_char_input)
        c_char_embed = tf.nn.embedding_lookup(char_combined, c_char_input)
        
        squeeze_to_word_q = tf.layers.max_pooling1d(q_char_embed, 16, 16)
        squeeze_to_word_q = tf.nn.dropout(squeeze_to_word_q, 1-dp*0.5)
        squeeze_to_word_c = tf.layers.max_pooling1d(c_char_embed, 16, 16)
        squeeze_to_word_c = tf.nn.dropout(squeeze_to_word_c, 1-dp*0.5)
        
    with tf.variable_scope("embedding_output"):
        q_embed_out = tf.concat([q_embed, squeeze_to_word_q], 2)
        c_embed_out = tf.concat([c_embed, squeeze_to_word_c], 2)
        q_embed_out = highway("highway", q_embed_out)
        c_embed_out = highway("highway", c_embed_out)

with tf.variable_scope("Embedding_Encoder_Layer"):
    # embedding encoder layer
    q_encoded = embedding_encoder_block("encoder_block", q_embed_out)
    c_encoded = embedding_encoder_block("encoder_block", c_embed_out)
    print(q_encoded.shape, c_encoded.shape)
    
with tf.variable_scope("Context_Query_Attention_Layer"):
    # context_query attention layer
    # first compute similarity matrix between context and query
    # S_tj = w * [C_t; Q_j; C_t*Q_j]
    c_expand = tf.expand_dims(c_encoded, 2)
    c_expand = tf.tile(c_expand, [1,1,30,1])
    
    q_expand = tf.expand_dims(q_encoded, 1)
    q_expand = tf.tile(q_expand, [1,240,1,1])
    
    qc_mul = tf.multiply(c_expand, q_expand)
    
    qc_concat = tf.concat([c_expand,q_expand,qc_mul], 3)
    w = tf.get_variable("s_w", [384,1])
    tf.summary.histogram("S_matrix_weight", w)
    
    A, B = query_context_co_attention(w, qc_concat, c_encoded, q_encoded)
    
    # layer output
    G = tf.concat([c_encoded, A, tf.multiply(c_encoded,A), tf.multiply(c_encoded,B)],2)
    print(G.shape)

with tf.variable_scope("Model_Encoder_Layer"):
    # model encoder layer
    model_encoder1 = model_encoder_block("model_encoder", G, projection=True)
    model_encoder2 = model_encoder_block("model_encoder", model_encoder1, projection=False)
    model_encoder3 = model_encoder_block("model_encoder", model_encoder2, projection=False)
    
    print(model_encoder1.shape,model_encoder2.shape,model_encoder3.shape)

with tf.variable_scope("Output_Layer"):
    # output layer
    # p1: start probability sequence
    # p2: end probability sequence
    p1_input = tf.concat([model_encoder1, model_encoder2],2)
    p2_input = tf.concat([model_encoder2, model_encoder3],2)
    
    l_input = tf.concat([model_encoder1, model_encoder2, model_encoder3],2)
    out_1 = tf.layers.separable_conv1d(l_input, 300, 1)
    out_2 = tf.squeeze(tf.layers.separable_conv1d(l_input, 1, 1),-1)
    out1 = tf.layers.dense(out_2,20,activation = tf.nn.relu)
    out_prob = tf.layers.dense(out1,classes,activation = tf.nn.softmax)
    out_labels = tf.argmax(out_prob,axis=1)
    
    print (out1.shape)
    
        
tf.add_to_collection("labels", out_labels)

global_step = tf.Variable(0,dtype=tf.int64,trainable=False,name='global_step')
    
with tf.variable_scope("Optimizer"):
    
    # add l2 weight decay to all variables
    trainables = tf.trainable_variables()
    loss_l2 = tf.add_n([ tf.nn.l2_loss(v) for v in trainables if 'bias' not in v.name ]) * 3e-7
    tf.summary.histogram("l2_loss", loss_l2)
    
    true_prob = tf.one_hot(labels,depth=classes)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=true_prob,logits=out_prob)) + loss_l2
    print (out_prob.shape,true_prob.shape)
    
    # perform cold warm up and gradient clipping
    lr = 1e-7
    lr = tf.minimum(lr, lr / tf.log(999.) * tf.log(tf.cast(global_step, tf.float32) + 1))
    optimizer = tf.train.AdamOptimizer(lr, beta1=0.8,epsilon=1e-7)
    opt_op = optimizer.minimize(loss, global_step=global_step)

    # apply exponential moving average
    # used in inference
    ema = tf.train.ExponentialMovingAverage(decay=0.9999)
    with tf.control_dependencies([opt_op]):
        train_step = ema.apply(trainables)

tf.add_to_collection("train_step", train_step)
        
tf.summary.scalar("loss", loss)

f_measure_train = tf.get_variable("f_train", (), trainable=False)
f_measure_dev = tf.get_variable("f_dev", (), trainable=False)
tf.summary.scalar("f_train", f_measure_train)
tf.summary.scalar("f_dev", f_measure_dev)
print(out_labels.shape)

In [ ]:
#with tf.device("/device:GPU:0"):

config = tf.ConfigProto()
config.allow_soft_placement = True
config.gpu_options.allow_growth = True

sess = tf.Session(config=config)

sess.run(tf.global_variables_initializer())
sess.run(train_iter.initializer)
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter("log/", sess.graph)
saver = tf.train.Saver(max_to_keep=2000)
gen = dev_batch()

cnt = 0
f_t = 0
f_d = 0

In [ ]:

while True:
    try:

        next_c, next_c_char, next_q, next_q_char, next_labels = sess.run(next_batch)
        
        feed_dict = {q_input: next_q,
                     q_char_input: next_q_char,
                     c_input: next_c,
                     c_char_input: next_c_char,
                     labels:next_labels,
                     f_measure_train: f_t,
                     f_measure_dev: f_d,
                     batch_size: len(next_c),
                     dp:0.5}

        run_ops = [train_step, out_labels, global_step, merged]

        if cnt % 5000 == 0:
            run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
            run_metadata = tf.RunMetadata()
            _, pl, step, s = sess.run(run_ops, feed_dict=feed_dict, options=run_options, run_metadata=run_metadata)
            writer.add_run_metadata(run_metadata, "steps{}".format(step), global_step=step)
            writer.add_summary(s, step)
            fetched_timeline = timeline.Timeline(run_metadata.step_stats)
            chrome_trace = fetched_timeline.generate_chrome_trace_format()
            with open('timeline/timeline{}.json'.format(step), 'w') as f:
                f.write(chrome_trace)
        else:
            _, pl, step, s = sess.run(run_ops, feed_dict=feed_dict)
            writer.add_summary(s, step)
            
        if cnt % 10 == 0:
            f_t = (np.round(pl).astype(int)==next_labels).mean()

        if cnt % 10 == 0:
            c_d, c_char_d, q_d, q_char_d, dev_label = next(gen)
            pl = sess.run(out_labels, feed_dict={q_input:q_d,
                                                             q_char_input: q_char_d,
                                                             c_input: c_d,
                                                             c_char_input: c_char_d,
                                                             labels:dev_label,
                                                             dp: 0})
            f_d = (np.round(pl).astype(int)==dev_label).mean()



        if cnt % 5000 == 0:
            print(cnt)
            saver.save(sess, "model2/strong", global_step=step)

        print (cnt,end = '\r')
        cnt += 1
    except tf.errors.OutOfRangeError:
        saver.save(sess, "model1/strong", global_step=step)
print("done!")
